파일 유형으로 파일 분류하기

In [10]:
# 필요한 라이브러리 들여오고 훈련과 테스트를 위해 사용할 샘플의 경로를 지정하는 것부터 시작
import os
from sklearn.feature_extraction.text import HashingVectorizer, TfidfTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.pipeline import Pipeline

javascript_path = "JavascriptSamples/"
python_path = "PythonSamples/"
powershell_path = "PowerShellSamples/"

In [11]:
# 세 유형의 모든 파일을 읽고, javascript/python/powershell 레이블을 각각 -1, 0, 1로 표현하는 배열을 만듦
corpus = []
labels = []
file_types_and_labels = [(javascript_path, -1), (python_path, 0), (powershell_path, 1)]
for files_path, label in file_types_and_labels:
    files = os.listdir(files_path)
    for file in files:
        file_path = files_path + "/" + file
        try:
            with open(file_path, "r") as myfile:
                data = myfile.read().replace("\n", "")
        except:
            pass
        data = str(data)
        corpus.append(data)
        labels.append(label)

In [15]:
print(corpus[0:5])
print(labels[0:5])

["// use this file with runners like node-debug// or mocha.require('babel-register');var app = require('./server');app.start();", "'use strict';var exclusionRegex = /^index|examples\\/|ptore2e\\//;module.exports = function createSitemap() {  return {    $runAfter: ['paths-computed'],    $runBefore: ['rendering-docs'],    $process: function(docs) {      docs.push({        id: 'sitemap.xml',        path: 'sitemap.xml',        outputPath: '../sitemap.xml',        template: 'sitemap.template.xml',        urls: docs.filter(function(doc) {          return doc.path &&            doc.outputPath &&            !exclusionRegex.test(doc.outputPath);        }).map(function(doc) {          return doc.path;        })      });    }  };};", '\'use strict\';const common = require(\'../common\');if (common.isWindows) {  common.skip(\'`make doc` does not run on Windows\');}// This tests that `make doc` generates the documentation properly.// Note that for this test to pass, `make doc` must be run first.co

In [12]:
# 파일에 대해 기본 자연어 처리를 수행할 훈련 및 테스트 데이터와 파이프라인을 만들고 랜덤 포레스트 분류기를 생성
X_train, X_test, y_train, y_test = train_test_split(
    corpus, labels, test_size=0.33, random_state=11
)
text_clf = Pipeline(
    [
        ("vect", HashingVectorizer(input="content", ngram_range=(1, 3))),
        ("tfidf", TfidfTransformer(use_idf=True,)),
        ("rf", RandomForestClassifier(class_weight="balanced")),
    ]
)

In [13]:
# 파이프라인을 훈련 데이터에 적합하고, 이를 이용하여 테스트 데이터를 예측 후 정확도와 혼돈행렬을 출력한다.
text_clf.fit(X_train, y_train)
y_test_pred = text_clf.predict(X_test)
print(f"정확도: {accuracy_score(y_test, y_test_pred) * 100 :.4f}%")
print(confusion_matrix(y_test, y_test_pred))

정확도: 99.2014%
[[1222    0    0]
 [  14  516    0]
 [   0    0    1]]
